# Distinct Pairwise Sum Sequence

Consider a sequence of 10 distinct positive intergers. Each pair of them has a distinct sum. What is the smallest possible value of the largest number of such sequence?

## Generate One

It is not hard to generate one such sequence using greedy search.
Such as the first 10 numbers of sequence [A011185](https://oeis.org/A011185).

In [7]:
# increasing sequence such that each pair of numbers has a distinct sum

def generator():
    a, numbers, sums = 1, [], set()
    while True:
        for i in numbers:
            if a + i in sums:
                break
        else:
            for i in numbers:
                sums.add(a + i)
            numbers.append(a)
            yield a
        a += 1

g = generator()
print([next(g) for _ in range(10)])

[1, 2, 3, 5, 8, 13, 21, 30, 39, 53]


## Verification

But is $53$ the answer to the initial question? Maybe not.
We should iterate all possible sequences with largest number $52$ to verify it.

In [4]:
# confirm there is no smaller solution

import itertools

def check_valid(sequence):
    sums = set()
    for i in range(10):
        for j in range(i + 1, 10):
            if sequence[i] + sequence[j] in sums:
                return False
            sums.add(sequence[i] + sequence[j])
    return True

for sequence in itertools.combinations(range(1, 53), 10):
    if check_valid(sequence):
        print(sequence)
        break
else:
    print('no smaller solution')

(1, 2, 3, 5, 8, 14, 23, 33, 41, 49)


## Best Result

The program above would take some time to run. Because there are $1.5\times 10^{10}$ such combinations. But it's OK.
It took my computer about 30 seconds to find a better solution.

If we remove the `break` in the program above, it shows multiple results. Among which we can see even smaller result.
Let's modify our program and find the best one.

The key is to generate combination sequence with increasing largest numbers.

Also, the current algorithm makes a set `sums` of 45 elements to verify each sequence. We should find a faster way.

We notice that if a sequence $\{a_n\}$ is a sequence with distinct pairwise sum, $\{a_n + m\}$ is also a sequence with distinct pairwise sum. Therefore, a unique distinct-pairwise-sum sequence can be represented by the difference between each element and its first element. 

Let $\{b_n\} = \{a_{n + 1} - a_1\}, (1\le n \lt N)$. E.g. for $\{a_n\} = \{1, 2, 3, 5\}$, $\{b_n\} = \{ 1, 2, 4 \}$. So that another sequence starting with $3$: $\{3, 4, 5, 7\}$ is also a distinct-pairwise-sum sequence.

Then we notice that $\{b_n\}$ should also have distinct pairwise sum. Otherwise if some $b_i + b_j = b_k + b_l, (1\le i,j,k,l \lt N)$, we would have $a_{i+1} + a_{j+1} = a_{k+1} + a_{l+1}, (1\le i,j,k,l \le N)$.

So if we find a sequence $\{b_n\}$ of length $N - 1$, we can find many sequences $\{a_n\}$ of length $N$.

If we limit the largest number of $a_n \le A$, we can find all distinct-pairwise-sum sequences of length $N$:
- Starting from $N = 0$, the only distinct-pairwise-sum sequences of length $0$ with largest number $\le A-N$ is $\{\}$.
- If we found all distinct-pairwise-sum sequences of length $n$ with largest number $\le A-N+n$, adding a number $a_1 \ge 1$ would make a new distinct-pairwise-sum sequences of length $n + 1$ with largest number $\le A-N+n+1$. There are about $A - N + n$ such new sequences (to garantee the largest number do not exceed).

To check whether the newly created sequence is distinct-pairwise-sum sequence, we already checked pairwise sums for $b_n$, so we just want to make sure pairs with new $a_1$ would not have duplicated sum:
$$ a_1 + (b_i + a_1) \notin \{ (b_j + a_1) + (b_k + a_1) \} $$
which can be simplified to
$$ b_i \notin \{ b_j + b_k \} $$

We should also notice that, if some $a_i + a_j = a_k + a_l$, we have $a_i - a_k = a_l - a_j$. Therefore, a distinct-pairwise-sum sequence of length $10$ must have $45$ distinct pairwise differences. So the smallest possible solution should be $46$. We can start with $A=46$ to find best solution.

In [49]:
# find best solution

def check_valid(sequence, sums):
    for ai in sequence:
        # if a1 + new_ai in new_sums
        # i.e. a1 + (ai + a1) = (aj + a1) + (ak + a1)
        # then ai in sums
        if ai in sums:
            return False
    return True

A = 46
N = 10
sequences = [([i], set()) for i in range(1, A - N + 2)]
print('n =', 1, ', largest num =', A - N + 1, ', num of sequences =', len(sequences))
for n in range(2, N + 1):
    max_num = A - N + n
    new_sequences = []
    for sequence, sums in sequences:
        if check_valid(sequence, sums):
            for a1 in range(1, max_num - sequence[-1] + 1):
                new_sequence = [a1] + [ai + a1 for ai in sequence]
                new_sums = {s + a1 + a1 for s in sums} | {ai + a1 + a1 for ai in sequence}
                new_sequences.append((new_sequence, new_sums))
    sequences = new_sequences
    print('n =', n, ', largest num =', A - N + n, ', num of sequences =', len(sequences))

if sequences:
    print('Best solutions:')
    for sequence, sums in sequences:
        print('Sequence:', sequence)
        print('Sums:', sums)
else:
    print('No solution')


n = 1 , largest num = 37 , num of sequences = 37
n = 2 , largest num = 38 , num of sequences = 703
n = 3 , largest num = 39 , num of sequences = 9139
n = 4 , largest num = 40 , num of sequences = 86640
n = 5 , largest num = 41 , num of sequences = 571608
n = 6 , largest num = 42 , num of sequences = 2240888
n = 7 , largest num = 43 , num of sequences = 3742288
n = 8 , largest num = 44 , num of sequences = 1462854
n = 9 , largest num = 45 , num of sequences = 50520
n = 10 , largest num = 46 , num of sequences = 2
Best solutions:
Sequence: [1, 3, 5, 20, 25, 33, 36, 39, 45, 46]
Sums: {4, 6, 8, 21, 23, 25, 26, 28, 30, 34, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 56, 58, 59, 61, 64, 65, 66, 69, 70, 71, 72, 75, 78, 79, 81, 82, 84, 85, 91}
Sequence: [1, 2, 8, 11, 14, 22, 27, 42, 44, 46]
Sums: {3, 9, 10, 12, 13, 15, 16, 19, 22, 23, 24, 25, 28, 29, 30, 33, 35, 36, 38, 41, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 54, 55, 56, 57, 58, 60, 64, 66, 68, 69, 71, 73, 86, 88, 90}


## Solution

We found 2 solutions with largest number to be $46$. Great!